In [148]:
# Re-import necessary modules after kernel reset
import pandas as pd
import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
import psycopg2
import logging
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, root_mean_squared_error
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, window, first, lag, lead
from pyspark.sql.window import Window
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.evaluation import RegressionMetrics

In [149]:
# Re-define the core functions due to kernel reset
def connect_to_postgres():
    try:
        conn = psycopg2.connect(
            dbname="airflow",
            user="airflow",
            password="airflow",
            host="localhost",
            port="5432"
        )
        return conn
    except Exception as e:
        logging.error(f"❌ Failed to connect to PostgreSQL: {e}")
        return None
    
def load_data_from_postgres(symbol):
    conn = connect_to_postgres()
    if conn:
        query = f"SELECT * FROM crypto_data WHERE symbol = '{symbol}';"
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    return None

In [150]:
# ✅ Remove outliers using IQR method
def remove_outliers_iqr(df, columns):
    for col_name in columns:
        Q1 = df[col_name].quantile(0.25)
        Q3 = df[col_name].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        before = len(df)
        df = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
        after = len(df)
        print(f"📉 Removed {before - after} outliers from '{col_name}'")
    return df

In [151]:
def prepare_timeseries_data(spark_df: DataFrame,
                             start: datetime.datetime = datetime.datetime(2023, 1, 1, 0, 0, 0),
                             end: datetime.datetime = datetime.datetime(2025, 10, 1, 0, 0, 0),
                             window_duration: str = "12 hours",
                             lag_days: int = 7) -> DataFrame:   # ✅ Đổi tên biến tại đây

    # Bước 1: Lọc theo khoảng thời gian
    spark_df = spark_df.filter((col("time") >= start) & (col("time") <= end))
    
    # Bước 2: Gom nhóm theo cửa sổ thời gian
    spark_df = spark_df.groupBy(
        window(col("time"), window_duration)
    ).agg(
        *[first(col_name).alias(col_name) for col_name in spark_df.columns if col_name != "time"]
    )

    # Bước 3: Thêm cột thời gian đại diện nhóm
    spark_df = spark_df.withColumn("time_group", col("window.start")).drop("window")

    # Bước 4: Tạo đặc trưng lịch sử
    window_spec = Window.orderBy("time_group")
    for i in range(1, lag_days + 1):    # ✅ Dùng lại biến mới tại đây
        for col_name in ["open", "high", "low", "close", "volume"]:
            spark_df = spark_df.withColumn(f"{col_name}_b_{i}", lag(col_name, i).over(window_spec))

    # Bước 5: Loại bỏ các hàng thiếu dữ liệu lịch sử
    spark_df = spark_df.dropna()

    # Bước 6: Tạo nhãn NEXT_CLOSE
    spark_df = spark_df.withColumn("NEXT_CLOSE", lead("close", 1).over(window_spec))

    # Bước 7: Loại bỏ hàng cuối cùng không có nhãn
    spark_df = spark_df.dropna()

    # Bước 8: Loại bỏ các cột không cần thiết
    spark_df = spark_df.drop("symbol", "time", "time_group")

    return spark_df

In [152]:
def split_data(spark_df, prediction_days=75):
    total_rows = spark_df.count()
    train_rows = total_rows - prediction_days
    train_df = spark_df.limit(train_rows)
    test_df = spark_df.subtract(train_df)
    return train_df, test_df

def transform_to_labeledpoint(df: DataFrame):
    return df.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))

In [153]:
def plot_train_test_metrics(metrics, model_name="Model"):
    plt.figure(figsize=(12, 8))

    num_iterations = len(metrics["mse_train"])  # Số lượng iterations
    x_axis = range(0, num_iterations)

    plt.plot(x_axis, metrics["mse_train"], label=f"Train MSE")
    plt.plot(x_axis, metrics["mse_test"], label=f"Test MSE")
    plt.plot(x_axis, metrics["rmse_train"], label=f"Train RMSE")
    plt.plot(x_axis, metrics["rmse_test"], label=f"Test RMSE")
    plt.plot(x_axis, metrics["mape_train"], label=f"Train MAPE")
    plt.plot(x_axis, metrics["mape_test"], label=f"Test MAPE")
    plt.plot(x_axis, metrics["r2_train"], label=f"Train R2")
    plt.plot(x_axis, metrics["r2_test"], label=f"Test R2")

    plt.title(f'{model_name} Metrics Over Iterations')
    plt.xlabel("Iterations")
    plt.ylabel("Metric Value")
    plt.legend()
    plt.show()

In [154]:
from sklearn.ensemble import RandomForestRegressor
def train_random_forest(train_df, test_df, plot=True):
    train_pd = train_df.toPandas()
    test_pd = test_df.toPandas()
    X_train = train_pd.drop(columns=["NEXT_CLOSE"])
    y_train = train_pd["NEXT_CLOSE"]
    X_test = test_pd.drop(columns=["NEXT_CLOSE"])
    y_test = test_pd["NEXT_CLOSE"]

    model = RandomForestRegressor(n_estimators=100, max_depth=6)

    # Train model
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    mse_train = mean_squared_error(y_train, train_preds)
    mse_test = mean_squared_error(y_test, test_preds)
    rmse_train = mse_train ** 0.5
    rmse_test = mse_test ** 0.5
    mape_train = mean_absolute_percentage_error(y_train, train_preds)
    mape_test = mean_absolute_percentage_error(y_test, test_preds)
    r2_train = r2_score(y_train, train_preds)
    r2_test = r2_score(y_test, test_preds)

    metrics = {
        "mse": (mse_train, mse_test),
        "rmse": (rmse_train, rmse_test),
        "mape": (mape_train, mape_test),
        "r2": (r2_train, r2_test)
    }

    # if plot:
    #     plot_train_test_metrics(metrics, model_name="Random Forest")

    return metrics

In [155]:
def evaluate_random_forest_model_train_test(model, train_rdd, test_rdd, plot=True):
    train_preds = list(model.predict(train_rdd.map(lambda x: x.features)).collect())
    train_labels = train_rdd.map(lambda x: x.label).collect()

    test_preds = list(model.predict(test_rdd.map(lambda x: x.features)).collect())
    test_labels = test_rdd.map(lambda x: x.label).collect()

    metrics = {
        "mse": (
            mean_squared_error(train_labels, train_preds),
            mean_squared_error(test_labels, test_preds)
        ),
        "rmse": (
            root_mean_squared_error(train_labels, train_preds, squared=False),
            root_mean_squared_error(test_labels, test_preds, squared=False)
        ),
        "mape": (
            mean_absolute_percentage_error(train_labels, train_preds),
            mean_absolute_percentage_error(test_labels, test_preds)
        ),
        "r2": (
            r2_score(train_labels, train_preds),
            r2_score(test_labels, test_preds)
        )
    }

    # if plot:
    #     plot_train_test_metrics(metrics, model_name="Random Forest")

    return metrics


In [156]:
from sklearn.linear_model import LinearRegression

def train_linear_regression(train_df, test_df):
    train_pd = train_df.toPandas()
    test_pd = test_df.toPandas()
    X_train = train_pd.drop(columns=["NEXT_CLOSE"]).values  # Chuyển sang numpy array
    y_train = train_pd["NEXT_CLOSE"].values  # Chuyển sang numpy array
    X_test = test_pd.drop(columns=["NEXT_CLOSE"]).values  # Chuyển sang numpy array
    y_test = test_pd["NEXT_CLOSE"].values  # Chuyển sang numpy array

    model = LinearRegression()

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên train và test set
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    metrics = {
        "mse": (
            mean_squared_error(y_train, train_preds),
            mean_squared_error(y_test, test_preds)
        ),
        "rmse": (
            root_mean_squared_error(y_train, train_preds),
            root_mean_squared_error(y_test, test_preds)
        ),
        "mape": (
            mean_absolute_percentage_error(y_train, train_preds),
            mean_absolute_percentage_error(y_test, test_preds)
        ),
        "r2": (
            r2_score(y_train, train_preds),
            r2_score(y_test, test_preds)
        )
    }

    return metrics


In [157]:
import xgboost as xgb

def train_xgboost(train_df, test_df, plot=True):
    train_pd = train_df.toPandas()
    test_pd = test_df.toPandas()
    X_train = train_pd.drop(columns=["NEXT_CLOSE"]).values
    y_train = train_pd["NEXT_CLOSE"].values
    X_test = test_pd.drop(columns=["NEXT_CLOSE"]).values
    y_test = test_pd["NEXT_CLOSE"].values

    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        objective="reg:squarederror"
    )

    evals_result = {}  # Dictionary để lưu trữ kết quả đánh giá

    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric=["rmse", "mse", "mape", "r2"],  # Yêu cầu tính toán các metrics
        callbacks=[xgb.callback.record_evaluation(evals_result)], # Ghi lại kết quả vào biến evals_result
        verbose=False
    )

    metrics = {
        "mse_train": evals_result["validation_0"]["mse"],
        "mse_test": evals_result["validation_1"]["mse"],
        "rmse_train": evals_result["validation_0"]["rmse"],
        "rmse_test": evals_result["validation_1"]["rmse"],
        "mape_train": evals_result["validation_0"]["mape"],
        "mape_test": evals_result["validation_1"]["mape"],
        "r2_train": evals_result["validation_0"]["r2"],
        "r2_test": evals_result["validation_1"]["r2"],
    }

    return model, metrics

In [ ]:
# Proceed to execute the training for 10 crypto symbols
spark = SparkSession.builder.appName("CryptoPredict").getOrCreate()

symbols = ["BNBUSDT", "BTCUSDT", "ETHUSDT", "XRPUSDT", "SOLUSDT",
           "LTCUSDT", "ETCUSDT", "PEPEUSDT", "DOGEUSDT", "ADAUSDT"]

# Đoạn mã train_all_models_for_one_symbol
def train_all_models_for_one_symbol(symbol: str):
    print(f"\n🚀 Training models for: {symbol}")
    df = load_data_from_postgres(symbol)
    if df is None or df.empty:
        print(f"⚠️ Skipping {symbol} due to empty data.")
        return None
    df = remove_outliers_iqr(df, ["close"])

    # 💡 Auto-scale small value coins
    price_median = df["close"].median()
    scale_factor = 1.0
    if price_median < 0.01:
        scale_factor = 1e6
        print(f"🔧 Scaling {symbol} values by {scale_factor} due to small price.")
        for col_name in ["open", "high", "low", "close"]:
            df[col_name] = df[col_name] * scale_factor
        df["volume"] = df["volume"]  # không scale volume
    else:
        print(f"✅ No scaling needed for {symbol} (median close = {price_median})")

    spark_df = spark.createDataFrame(df)
    processed_df = prepare_timeseries_data(spark_df)
    train_df, test_df = split_data(processed_df)

    results = {}

    try:
        metrics = train_linear_regression(train_df, test_df)
        results["linear_regression"] = metrics
    except Exception as e:
        print(f"❌ Linear Regression failed for {symbol}: {e}")
        results["linear_regression"] = {"rmse": None, "r2": None, "mse": None, "mape": None}

    try:
        metrics = train_xgboost(train_df, test_df)
        results["xgboost"] = metrics
    except Exception as e:
        print(f"❌ XGBoost failed for {symbol}: {e}")
        results["xgboost"] = {"rmse": None, "r2": None, "mse": None, "mape": None}

    try:
        metrics = train_random_forest(train_df, test_df)
        results["random_forest"] = metrics
    except Exception as e:
        print(f"❌ Random Forest failed for {symbol}: {e}")
        results["random_forest"] = {"rmse": None, "r2": None, "mse": None, "mape": None}

    return {"symbol": symbol, "results": results}

# In kết quả
for sym in symbols:
    result = train_all_models_for_one_symbol(sym)
    if result:
        for model, metrics in result["results"].items():
            print(f"Model: {model}, Symbol: {result['symbol']}, "
                  f"RMSE: {metrics['rmse']}, MSE: {metrics['mse']}, "
                  f"MAPE: {metrics['mape']}, R2: {metrics['r2']}")


In [ ]:
def predict_next_close(symbol: str):
    print(f"\n🔮 Predicting next CLOSE for: {symbol}")
    df = load_data_from_postgres(symbol)
    print(df.head())
    print(f"📊 Loaded data for {symbol}: {len(df)} rows")
    if df is None or len(df) < 8:
        print(f"⚠️ Not enough data for prediction: {symbol}")
        return None

    df = df.sort_values("time", ascending=False).head(10000).sort_values("time")
    df = remove_outliers_iqr(df, ["close"])

    # Dựa vào các mô hình đã train trước đó, đưa ra dự đoán cho giá CLOSE tiếp theo bằng cả 3 mô hình
    # 1. Linear Regression không cần fit lại
    linear = train_linear_regression(df, df)
    linear_model = LinearRegression()
    linear_model.fit(df.drop(columns=["NEXT_CLOSE"]).values, df["NEXT_CLOSE"].values)


In [ ]:
result = predict_next_close("BTCUSDT")
print(result)


🚀 Training models for: BTCUSDT with anti-overfitting techniques


C:\Users\hoain\AppData\Local\Temp\ipykernel_6588\59782643.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


📉 Removed 0 outliers from 'close'


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hoain\\AppData\\Local\\Temp\\spark-51e85a69-8968-4163-a5fc-a687b3b91632\\pyspark-4cf919fa-428c-4088-9e01-3cc48d0bfd87\\tmpk30vednf'

In [ ]:
# # Import Libraries
# import numpy as np  # to handle matrix
# import pandas as pd # to handle data
# from matplotlib import pyplot as plt # to visualize
# import datetime, pytz # to handle time
# from sklearn.model_selection import train_test_split # Split data
# from sklearn.ensemble import RandomForestRegressor # Random Forest Classifier
# import logging
# import psycopg2
# import logging
# import pandas as pd
# import psycopg2
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, lag, avg
# from pyspark.sql.window import Window
# from pyspark.ml.feature import VectorAssembler, StandardScaler
# from pyspark.ml.regression import GBTRegressor
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml import Pipeline
# from pyspark.sql.functions import abs as sql_abs

In [ ]:
# def connect_to_postgres():
#     try:
#         conn = psycopg2.connect(
#             dbname="airflow",
#             user="airflow",
#             password="airflow",
#             host="localhost",
#             port="5432"
#         )
#         logging.info("✅ Connected to PostgreSQL.")
#         return conn
#     except Exception as e:
#         logging.error(f"❌ Failed to connect to PostgreSQL: {e}")
#         return None

In [ ]:
# # Tạo Spark session và connect to PostgreSQL bằng cách chạy hàm connect_to_postgres
# spark = SparkSession.builder.master("local").appName("Crypto").getOrCreate()
# conn = connect_to_postgres()
# # Nếu kết nối thành công, in ra dataframe từ bảng crypto_prices
# if conn:
#     # Read data from PostgreSQL
#     query = "SELECT * FROM crypto_data WHERE symbol = 'BNBUSDT';"
#     df = pd.read_sql(query, conn)
#     # Convert the DataFrame to a Spark DataFrame
#     spark_df = spark.createDataFrame(df)
#     # Close the connection
#     conn.close()
#     # In ra 5 dòng đầu tiên của dataframe
#     spark_df.show(5)

In [ ]:
# # In ra shape của dataframe
# print(f"Shape of DataFrame: {spark_df.count()} rows, {len(spark_df.columns)} columns")

In [ ]:
# from pyspark.sql.functions import col, window, first
# import datetime

# start = datetime.datetime(2023, 1, 1, 0, 0, 0)
# end = datetime.datetime(2025, 10, 1, 0, 0, 0)

# # Filter theo khoảng thời gian
# spark_df = spark_df.filter((col("time") >= start) & (col("time") <= end))

# # Group by theo cửa sổ thời gian 1 giờ
# spark_df = spark_df.groupBy(
#     window(col("time"), "3 hours")
# ).agg(
#     *[first(col_name).alias(col_name) for col_name in spark_df.columns if col_name != "time"]
# )

# # Nếu muốn hiển thị thời gian bắt đầu mốc group
# spark_df = spark_df.withColumn("time_group", col("window.start")).drop("window")

# # Hiển thị 3 dòng đầu
# spark_df.show(3)


In [ ]:
# from pyspark.sql.functions import lag
# from pyspark.sql.window import Window

# # Xác định cửa sổ để tính toán lag
# window_spec = Window.orderBy("time_group")

# # Thêm các cột lịch sử (lag features) cho 7 ngày
# for i in range(1, 8):  # for 7 days
#     spark_df = spark_df.withColumn(f"open_b_{i}", lag("open", i).over(window_spec))
#     spark_df = spark_df.withColumn(f"high_b_{i}", lag("high", i).over(window_spec))
#     spark_df = spark_df.withColumn(f"low_b_{i}", lag("low", i).over(window_spec))
#     spark_df = spark_df.withColumn(f"close_b_{i}", lag("close", i).over(window_spec))
#     spark_df = spark_df.withColumn(f"volume_b_{i}", lag("volume", i).over(window_spec))

# # Loại bỏ các hàng không đủ dữ liệu lịch sử
# spark_df = spark_df.dropna()

# # Hiển thị thông tin về dữ liệu sau khi thêm cột lịch sử
# print(f"Historical Data Shape: {spark_df.count()} rows, {len(spark_df.columns)} columns")
# spark_df.show(3)

In [ ]:
# from pyspark.sql.functions import lead
# from pyspark.sql.window import Window

# # Xác định cửa sổ để tính toán giá trị tiếp theo
# window_spec = Window.orderBy("time_group")

# # Thêm cột "NEXT_CLOSE" làm nhãn (label) cho giá trị đóng cửa tiếp theo
# spark_df = spark_df.withColumn("NEXT_CLOSE", lead("close", 1).over(window_spec))

# # Loại bỏ các hàng không đủ dữ liệu (hàng cuối cùng không có giá trị tiếp theo)
# spark_df = spark_df.dropna()

# # Loại bỏ cột symbol và time vì không cần thiết cho mô hình
# spark_df = spark_df.drop("symbol", "time", "time_group")


# # Hiển thị thông tin về dữ liệu sau khi thêm cột nhãn
# print(f"After adding NEXT_CLOSE Label, new shape: {spark_df.count()} rows, {len(spark_df.columns)} columns")
# spark_df.show(3)

In [ ]:
# # Số ngày dự đoán
# prediction_days = 250
# # Tổng số hàng trong DataFrame
# total_rows = spark_df.count()

# # Số hàng cho tập huấn luyện
# train_rows = total_rows - prediction_days

# # Chia dữ liệu thành tập train và test
# train_df = spark_df.limit(train_rows)  # Lấy `train_rows` đầu tiên làm tập huấn luyện
# test_df = spark_df.subtract(train_df)  # Phần còn lại là tập kiểm tra

# # Hiển thị thông tin về tập train/test
# print(f"PERCENT test/total data = {(prediction_days / total_rows) * 100:.2f}%")
# print(f"Train data shape: {train_df.count()} rows, {len(train_df.columns)} columns")
# print(f"Test data shape: {test_df.count()} rows, {len(test_df.columns)} columns")

In [ ]:
# spark_TrainingData = train_df
# spark_TestData = test_df

In [ ]:
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.linalg import Vectors

# transformed_TrainingData = spark_TrainingData.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
# transformed_TestData = spark_TestData.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))


In [ ]:
# print("Number of training set rows: %d" % transformed_TrainingData.count())
# print("Number of test set rows: %d" % transformed_TestData.count())

In [ ]:
# # Let's make sure we have the correct types.
# print("%s should be an RDD" % type(transformed_TrainingData))
# print("%s should be a LabeledPoint" % type(transformed_TestData.first()))

In [ ]:
# from pyspark.mllib.tree import RandomForest
# from time import *
# RF_NUM_TREES = 1000 # 5
# RF_MAX_DEPTH = 7 # 5
# IMPURITY ="variance"
# RF_MAX_BINS = 300
# RANDOM_SEED = 13579 #None

# start_time = time()

# model = RandomForest.trainRegressor(transformed_TrainingData, categoricalFeaturesInfo={}, \
#     numTrees=RF_NUM_TREES, featureSubsetStrategy="auto", impurity= IMPURITY, \
#     maxDepth=RF_MAX_DEPTH, maxBins=RF_MAX_BINS, seed=RANDOM_SEED)

# end_time = time()
# elapsed_time = end_time - start_time
# print("Time to train model: %.3f seconds" % elapsed_time)

In [ ]:
# # Make predictions
# predictions = model.predict(transformed_TestData.map(lambda x: x.features))

# # Repartition both RDDs to ensure they have the same number of partitions
# labels = transformed_TestData.map(lambda x: x.label).repartition(1)
# predictions = predictions.repartition(1)

# # Zip the RDDs after ensuring they are aligned
# labels_and_predictions = labels.zip(predictions)

In [ ]:
# # compute accuracy
# from pyspark.mllib.evaluation import RegressionMetrics

# start_time = time()

# metrics = RegressionMetrics(labels_and_predictions)
# print("Root Mean Squared Error: %.f" % (metrics.rootMeanSquaredError))
# print("R^2 Error: %.3f" % (metrics.r2))

# end_time = time()
# elapsed_time = end_time - start_time
# print("Time to evaluate model: %.3f seconds" % elapsed_time)

In [ ]:
# list_Real = transformed_TestData.map(lambda x: x.label).collect()
# list_Predicted = predictions.collect()

In [ ]:
# # Visualising the results
# plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
# ax = plt.gca()  
# # Convert test_df to Pandas DataFrame to access the index
# test_df_pd = test_df.toPandas()
# plt.plot(test_df_pd.index, list_Real, color='red', label='Real BTC Price')
# plt.plot(test_df_pd.index, list_Predicted, color='blue', label='Predicted BTC Price')
# plt.title('BTC Price Prediction', fontsize=40)
# x = test_df_pd.reset_index().index
# for tick in ax.xaxis.get_major_ticks():
#     tick.label1.set_fontsize(18)
# for tick in ax.yaxis.get_major_ticks():
#     tick.label1.set_fontsize(18)
# plt.xlabel('Time', fontsize=40)
# plt.ylabel('BTC Price(USD) [Closed]', fontsize=40)
# plt.legend(loc=2, prop={'size': 25})
# plt.show()